In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re
import time

# 影片設定
# 指定影片名稱
video_title = "加油！金順"
base_episode_url = "https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"
# 開始集數
start_episode = 1
# 結束集數（可調整為多集）
end_episode = 1 

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度，並確保包含影音
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
    'noplaylist': True                     # 如果是播放清單只下載單個影片
}

def extract_m3u8_url(page_content):
    # 使用正則表達式尋找 .m3u8 連結
    m3u8_regex = re.compile(r'https?://[^\s\'"]+\.m3u8')
    m3u8_url = m3u8_regex.search(page_content)
    if m3u8_url:
        return m3u8_url.group(0)
    return None

def download_video(video_url, episode_number):
    output_filename = f"{video_title}_第{episode_number}集.%(ext)s"
    try:
        # 嘗試直接下載影片，並指定集數為文件名稱
        with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
            ydl.download([video_url])
    except yt_dlp.utils.DownloadError:
        print(f"第 {episode_number} 集直接下載失敗，嘗試提取 .m3u8 串流鏈接...")

        # 使用 requests 和 BeautifulSoup 抓取網頁內容
        response = requests.get(video_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            m3u8_url = None

            # 嘗試從 script 標籤中提取 .m3u8 連結
            for script in soup.find_all('script'):
                m3u8_url = extract_m3u8_url(script.text)
                if m3u8_url:
                    break

            if m3u8_url:
                print(f"第 {episode_number} 集找到 .m3u8 串流鏈接: {m3u8_url}")
                try:
                    # 使用 m3u8 串流鏈接下載影片
                    with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                        ydl.download([m3u8_url])
                except yt_dlp.utils.DownloadError:
                    print(f"第 {episode_number} 集無法下載 .m3u8 串流鏈接。")
            else:
                print(f"第 {episode_number} 集未找到有效的 .m3u8 串流鏈接。")
        else:
            print(f"第 {episode_number} 集無法抓取網頁內容。")

# 下載範圍內的每一集
for episode_number in range(start_episode, end_episode + 1):
    episode_url = base_episode_url.format(episode_number=episode_number)
    print(f"下載 {video_title} 第 {episode_number} 集: {episode_url}")
    download_video(episode_url, episode_number)


In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re

# 影片設定
video_title = "加油！金順"
base_episode_url = "https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"
start_episode = 1
end_episode = 1 

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'merge_output_format': 'mp4',
    'noplaylist': True
}

def extract_m3u8_url(page_content):
    # 檢查各種位置可能的 .m3u8 連結
    m3u8_regex = re.compile(r'https?://[^\s\'"]+\.m3u8')
    
    # 1. 檢查頁面本身
    m3u8_url = m3u8_regex.search(page_content)
    if m3u8_url:
        return m3u8_url.group(0)
    
    # 2. 檢查 iframe 內嵌
    soup = BeautifulSoup(page_content, 'html.parser')
    for iframe in soup.find_all('iframe', src=True):
        iframe_src = iframe['src']
        if ".m3u8" in iframe_src:
            return iframe_src
        
        # 嘗試從 iframe URL 抓取內容
        iframe_content = requests.get(iframe_src).text
        m3u8_url = m3u8_regex.search(iframe_content)
        if m3u8_url:
            return m3u8_url.group(0)
    
    # 3. 檢查 script 標籤
    for script in soup.find_all('script'):
        m3u8_url = m3u8_regex.search(script.text)
        if m3u8_url:
            return m3u8_url.group(0)
    
    return None

def download_video(video_url, episode_number):
    output_filename = f"{video_title}_第{episode_number}集.%(ext)s"
    try:
        with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
            ydl.download([video_url])
    except yt_dlp.utils.DownloadError:
        print(f"第 {episode_number} 集直接下載失敗，嘗試提取 .m3u8 串流鏈接...")

        response = requests.get(video_url)
        if response.status_code == 200:
            m3u8_url = extract_m3u8_url(response.text)
            if m3u8_url:
                print(f"第 {episode_number} 集找到 .m3u8 串流鏈接: {m3u8_url}")
                try:
                    with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                        ydl.download([m3u8_url])
                except yt_dlp.utils.DownloadError:
                    print(f"第 {episode_number} 集無法下載 .m3u8 串流鏈接。")
            else:
                print(f"第 {episode_number} 集未找到有效的 .m3u8 串流鏈接。")
        else:
            print(f"第 {episode_number} 集無法抓取網頁內容。")

# 下載範圍內的每一集
for episode_number in range(start_episode, end_episode + 1):
    episode_url = base_episode_url.format(episode_number=episode_number)
    print(f"下載 {video_title} 第 {episode_number} 集: {episode_url}")
    download_video(episode_url, episode_number)


In [ ]:
import yt_dlp
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import time

# 影片設定
video_title = "加油！金順"
episode_number = 1
episode_url = f"https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"

# 設定 yt-dlp 的下載選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',
    'merge_output_format': 'mp4',
    'outtmpl': f"{video_title}_第{episode_number}集.%(ext)s"
}

def extract_avdb_url(page_source):
    # 使用正則表達式尋找以 avdb.me 開頭的重定向 URL
    avdb_regex = re.compile(r'https://vip\.avdb\.me/\?url=https://[^\s\'"]+\.m3u8')
    avdb_url = avdb_regex.search(page_source)
    if avdb_url:
        return avdb_url.group(0)
    return None

def download_video(avdb_url):
    # 使用 yt-dlp 下載影片
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([avdb_url])
        print(f"{video_title} 第 {episode_number} 集下載完成！")
    except yt_dlp.utils.DownloadError:
        print(f"無法下載 {video_title} 第 {episode_number} 集的 .m3u8 串流鏈接。")

# 設定 Selenium 的選項
chrome_options = Options()
chrome_options.add_argument("--headless")  # 啟用無頭模式
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 使用 selenium 自動下載並啟動 ChromeDriver
driver = webdriver.Chrome(options=chrome_options)

# 開啟目標頁面
driver.get(episode_url)
time.sleep(5)  # 等待頁面加載

# 提取網頁內容
page_source = driver.page_source
avdb_url = extract_avdb_url(page_source)

# 關閉瀏覽器
driver.quit()

# 檢查並下載影片
if avdb_url:
    print(f"找到重定向 .m3u8 連結: {avdb_url}")
    download_video(avdb_url)
else:
    print("未找到有效的重定向 .m3u8 連結。")


In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re
import opencc

# 影片設定
video_title_traditional = "加油！金順"  # 使用繁體中文影片名稱
start_episode = 1  # 開始集數
end_episode = 1    # 結束集數

# 將繁體中文轉為簡體中文（因網站為簡體中文）
converter = opencc.OpenCC('t2s')
video_title = converter.convert(video_title_traditional)

# 官網首頁 URL
base_url = "https://94itv.app/"
base_episode_url = "https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度，並確保包含影音
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
    'outtmpl': f"{video_title_traditional}_第{{episode_number}}集.%(ext)s"  # 設置下載文件名格式
}

def extract_avdb_url(page_content):
    """
    從網頁內容中提取並生成 avdb 的跳轉 url（m3u8 解析網址）
    """
    m3u8_regex = re.compile(r'https?://[^\s\'"]+\.m3u8')
    m3u8_url = m3u8_regex.search(page_content)
    if m3u8_url:
        return f"https://vip.avdb.me/?url={m3u8_url.group(0)}"
    return None

def download_video(video_url, episode_number):
    output_filename = f"{video_title_traditional}_第{episode_number}集.%(ext)s"
    try:
        # 嘗試直接下載影片，並指定集數為文件名稱
        with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
            ydl.download([video_url])
    except yt_dlp.utils.DownloadError:
        print(f"第 {episode_number} 集直接下載失敗，嘗試提取 .m3u8 串流鏈接...")

        # 使用 requests 和 BeautifulSoup 抓取網頁內容
        response = requests.get(video_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            avdb_url = None

            # 在網頁內容中查找 m3u8 串流的原始鏈接
            m3u8_link = extract_avdb_url(response.text)
            
            # 若找到 m3u8 原始鏈接，則轉換為 avdb 的解析鏈接
            if m3u8_link:
                avdb_url = m3u8_link

            if avdb_url:
                print(f"第 {episode_number} 集找到 avdb 解析鏈接: {avdb_url}")
                try:
                    # 使用 avdb 跳轉的 m3u8 下載影片
                    with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                        ydl.download([avdb_url])
                except yt_dlp.utils.DownloadError:
                    print(f"第 {episode_number} 集無法下載 avdb 解析的 m3u8 串流鏈接。")
            else:
                print(f"第 {episode_number} 集未找到有效的 avdb 跳轉鏈接。")
        else:
            print(f"第 {episode_number} 集無法抓取網頁內容。")

# 下載範圍內的每一集
for episode_number in range(start_episode, end_episode + 1):
    episode_url = base_episode_url.format(episode_number=episode_number)
    print(f"下載 {video_title_traditional} 第 {episode_number} 集: {episode_url}")
    download_video(episode_url, episode_number)


In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re
import opencc

# 影片設定
video_title_traditional = "加油！金順"  # 使用繁體中文影片名稱
start_episode = 1  # 開始集數
end_episode = 1    # 結束集數

# 將繁體中文轉為簡體中文（因網站為簡體中文）
converter = opencc.OpenCC('t2s')
video_title = converter.convert(video_title_traditional)

# 官網首頁 URL
base_url = "https://94itv.app/"
base_episode_url = "https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度，並確保包含影音
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
    'outtmpl': f"{video_title_traditional}_第{{episode_number}}集.%(ext)s"  # 設置下載文件名格式
}

def extract_avdb_url(page_content):
    """
    從網頁內容中提取並生成 avdb 的跳轉 url（m3u8 解析網址）
    """
    m3u8_regex = re.compile(r'https?://[^\s\'"]+\.m3u8')
    m3u8_url = m3u8_regex.search(page_content)
    if m3u8_url:
        return f"https://vip.avdb.me/?url={m3u8_url.group(0)}"
    return None

def download_video(video_url, episode_number):
    output_filename = f"{video_title_traditional}_第{episode_number}集.%(ext)s"
    try:
        # 嘗試直接下載影片，並指定集數為文件名稱
        with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
            ydl.download([video_url])
    except yt_dlp.utils.DownloadError:
        print(f"第 {episode_number} 集直接下載失敗，嘗試提取 .m3u8 串流鏈接...")

        # 使用 requests 和 BeautifulSoup 抓取網頁內容
        response = requests.get(video_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            avdb_url = None

            # 在網頁內容中查找 m3u8 串流的原始鏈接
            m3u8_link = extract_avdb_url(response.text)
            
            # 若找到 m3u8 原始鏈接，則轉換為 avdb 的解析鏈接
            if m3u8_link:
                avdb_url = m3u8_link

            if avdb_url:
                print(f"第 {episode_number} 集找到 avdb 解析鏈接: {avdb_url}")
                try:
                    # 使用 avdb 跳轉的 m3u8 下載影片
                    with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                        ydl.download([avdb_url])
                except yt_dlp.utils.DownloadError:
                    print(f"第 {episode_number} 集無法下載 avdb 解析的 m3u8 串流鏈接。")
            else:
                print(f"第 {episode_number} 集未找到有效的 avdb 跳轉鏈接。")
        else:
            print(f"第 {episode_number} 集無法抓取網頁內容。")

# 下載範圍內的每一集
for episode_number in range(start_episode, end_episode + 1):
    episode_url = base_episode_url.format(episode_number=episode_number)
    print(f"下載 {video_title_traditional} 第 {episode_number} 集: {episode_url}")
    download_video(episode_url, episode_number)


In [ ]:
import yt_dlp
import requests
from bs4 import BeautifulSoup
import re

# 影片設定
video_title = "加油！金順"
base_episode_url = "https://94itv.app/vod-play-id-207163-sid-2-nid-{episode_number}.html"
start_episode = 1
end_episode = 1  # 可以調整為所需集數

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度，並確保包含影音
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
    'outtmpl': f'{video_title}_第%(episode_number)d集.%(ext)s',  # 以集數命名
    'noplaylist': True                     # 如果是播放清單只下載單個影片
}

def extract_m3u8_url(video_url):
    # 使用 requests 抓取網頁內容
    response = requests.get(video_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        m3u8_url = None
        
        # 嘗試從 HTML 中提取 m3u8 連結
        for script in soup.find_all('script'):
            m3u8_url_match = re.search(r'https?://[^\s\'"]+\.m3u8', script.text)
            if m3u8_url_match:
                m3u8_url = m3u8_url_match.group(0)
                break
        
        # 返回提取到的 m3u8 URL
        return m3u8_url
    else:
        print(f"無法抓取網頁內容: {video_url}")
        return None

def download_video(video_url, episode_number):
    m3u8_url = extract_m3u8_url(video_url)
    if m3u8_url:
        print(f"第 {episode_number} 集找到 .m3u8 串流鏈接: {m3u8_url}")
        try:
            # 使用 m3u8 串流鏈接下載影片
            output_filename = f"{video_title}_第{episode_number}集.%(ext)s"
            with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
                ydl.download([m3u8_url])
            print(f"第 {episode_number} 集下載成功！")
        except yt_dlp.utils.DownloadError as e:
            print(f"第 {episode_number} 集下載失敗: {e}")
    else:
        print(f"第 {episode_number} 集未找到有效的 .m3u8 串流鏈接。")

# 下載範圍內的每一集
for episode_number in range(start_episode, end_episode + 1):
    episode_url = base_episode_url.format(episode_number=episode_number)
    print(f"下載 {video_title} 第 {episode_number} 集: {episode_url}")
    download_video(episode_url, episode_number)


In [ ]:
import yt_dlp

# 定義 m3u8 URL
urls = [
    # "https://cdn.wlcdn99.com:777/XP3h0tM8/index.m3u8",
    "https://cdn.wlcdn99.com:777/dKsyDRZM/index.m3u8"
]

# 設定 yt-dlp 的選項
ydl_opts = {
    'format': 'best',                   # 選擇最佳解析度
    'outtmpl': 'downloaded_video.%(ext)s',  # 設定輸出文件名
    'merge_output_format': 'mp4',       # 合併為 mp4 格式
}

# 測試下載
for url in urls:
    try:
        print(f"正在嘗試下載: {url}")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"下載成功: {url}")
    except yt_dlp.utils.DownloadError as e:
        print(f"下載失敗: {url} - {str(e)}")


In [ ]:
import yt_dlp

# 設定下載選項
ydl_opts = {
    'format': 'bestvideo+bestaudio/best',  # 選擇最高解析度，並確保包含影音
    'merge_output_format': 'mp4',          # 合併為 mp4 格式
    'outtmpl': '%(title)s.%(ext)s',        # 輸出的文件名
}

# 集數列表（這裡列出所有要下載的 .m3u8 連結）
urls = [
    "https://cdn.wlcdn99.com:777/XP3h0tM8/index.m3u8",
    "https://cdn.wlcdn99.com:777/dKsyDRZM/index.m3u8"
]

# 遍歷每個集數的 URL 進行下載
for idx, url in enumerate(urls, start=2):
    output_filename = f"加油！金順_第{idx}集.%(ext)s"
    print(f"正在嘗試下載：{url}")
    try:
        with yt_dlp.YoutubeDL({**ydl_opts, 'outtmpl': output_filename}) as ydl:
            ydl.download([url])
        print(f"第 {idx} 集下載完成，保存為 {output_filename}")
    except yt_dlp.utils.DownloadError:
        print(f"第 {idx} 集下載失敗：{url}")
